# Crumple Video - Blender XPBD Paper Animation

Генерация видеоанонса с эффектом раскрывающейся бумаги.

**Особенности:**
- Полная Blender XPBD физика на CPU
- Обложка с датами и типографикой
- Аудиотрек The xx - Intro (старт с 1:10)

**Версия v2.0 - с полными XPBD скриптами**

In [ ]:
NOTEBOOK_VERSION = 'v2.1-blender36'
print(f'🚀 Engine {NOTEBOOK_VERSION}')

# ==========================================
# 1. УСТАНОВКА ЗАВИСИМОСТЕЙ (Blender 3.6)
# ==========================================
import warnings
warnings.filterwarnings("ignore")

print("⏳ Installing dependencies...", flush=True)

# Install FFmpeg from apt
!apt-get update -qq && apt-get install -y -qq ffmpeg libxrender1 libxi6 libxkbcommon-x11-0 > /dev/null 2>&1

# Install Blender 3.6 LTS from tarball (compatible with XPBD script)
import os
if not os.path.exists('/opt/blender-3.6'):
    print('📦 Downloading Blender 3.6 LTS...', flush=True)
    !wget -q https://download.blender.org/release/Blender3.6/blender-3.6.16-linux-x64.tar.xz -O /tmp/blender.tar.xz
    !mkdir -p /opt && tar -xf /tmp/blender.tar.xz -C /opt
    !mv /opt/blender-3.6.* /opt/blender-3.6
    !ln -sf /opt/blender-3.6/blender /usr/local/bin/blender
    !rm /tmp/blender.tar.xz
else:
    print('✅ Blender 3.6 already installed')

# Verify Blender version
!blender --version | head -2

!pip install opencv-python numpy pillow requests > /dev/null

import os
import sys
import json
import glob
import shutil
import subprocess
from pathlib import Path

import numpy as np
import cv2
from PIL import Image, ImageFont, ImageDraw

def log(msg):
    print(msg, flush=True)

log("✅ Dependencies ready")


In [ ]:
# ==========================================
# 2. КОНФИГУРАЦИЯ
# ==========================================
KAGGLE_INPUT_ROOT = "/kaggle/input"
WORKING_DIR = Path("/kaggle/working")

# Ищем папки сессии
session_dirs = sorted(glob.glob(os.path.join(KAGGLE_INPUT_ROOT, "video-afisha-session-*")), reverse=True)
crumple_test_dirs = sorted(glob.glob(os.path.join(KAGGLE_INPUT_ROOT, "crumple-video-test*")), reverse=True)

if session_dirs:
    SOURCE_FOLDER = Path(session_dirs[0])
    log(f"✅ DETECTED BOT SESSION: {SOURCE_FOLDER}")
elif crumple_test_dirs:
    SOURCE_FOLDER = Path(crumple_test_dirs[0])
    log(f"✅ DETECTED TEST DATASET: {SOURCE_FOLDER}")
elif os.path.exists(os.path.join(KAGGLE_INPUT_ROOT, "afisha-dataset-2")):
    SOURCE_FOLDER = Path(os.path.join(KAGGLE_INPUT_ROOT, "afisha-dataset-2"))
    log(f"⚠️ Using legacy dataset: {SOURCE_FOLDER}")
else:
    all_dirs = [d for d in glob.glob(os.path.join(KAGGLE_INPUT_ROOT, "*")) if os.path.isdir(d)]
    SOURCE_FOLDER = Path(all_dirs[0]) if all_dirs else Path("/kaggle/input/afisha-dataset-2")
    log(f"⚠️ Fallback: {SOURCE_FOLDER}")

# Параметры видео
W, H = 1080, 1920  # Вертикальный формат
FPS = 24

# Аудио
AUDIO_START_SEC = 70  # 1:10 от начала трека
AUDIO_FILENAME = "The_xx_-_Intro.mp3"

# Blender params - больше кадров для плавности
FRAME_START = 1
FRAME_END = 64
BLENDER_SAMPLES = 48  # Более высокое качество

WORKING_DIR.mkdir(parents=True, exist_ok=True)
log("✅ Config ready")

In [ ]:
# ==========================================
# 3. ГЕНЕРАТОР ОБЛОЖКИ
# ==========================================

def find_font(name: str) -> str:
    """Ищет шрифт в SOURCE_FOLDER и стандартных путях."""
    candidates = [
        SOURCE_FOLDER / name,
        SOURCE_FOLDER / "fonts" / name,
        Path(f"/usr/share/fonts/truetype/{name}"),
    ]
    # Также ищем глобально
    candidates.extend(Path(KAGGLE_INPUT_ROOT).rglob(name))
    for p in candidates:
        if p.exists():
            return str(p)
    return None

def generate_cover_image(date_range: str, month: str, output_path: Path) -> Path:
    """
    Генерирует обложку в стиле макета:
    - Жёлтый фон #F5E10E
    - Даты крупно (Benzin-Bold)
    """
    # Размеры как в рендере Blender
    width, height = 864, 1104
    
    # Цвета
    bg_color = (245, 225, 14)  # #F5E10E жёлтый
    text_color = (16, 14, 14)  # Почти чёрный
    
    img = Image.new('RGB', (width, height), bg_color)
    draw = ImageDraw.Draw(img)
    
    # Загружаем шрифты
    font_benzin = find_font("Benzin-Bold.ttf")
    font_bebas = find_font("BebasNeue-Regular.ttf") or find_font("BebasNeue-Bold.ttf")
    font_oswald = find_font("Oswald-VariableFont_wght.ttf")
    font_druk = find_font("DrukCyr-Bold.ttf")
    
    def load_font(path, size):
        if path and os.path.exists(path):
            return ImageFont.truetype(path, size)
        return ImageFont.load_default()
    
    # Даты "25-27" - крупно сверху
    font_dates = load_font(font_benzin, 200)
    draw.text((50, 80), date_range, font=font_dates, fill=text_color)
    
    # "СОБЫТИЯ" - Bebas Neue
    font_events = load_font(font_bebas, 100)
    draw.text((350, 340), "СОБЫТИЯ", font=font_events, fill=text_color)
    
    # "КОТОРЫЕ" - Oswald, крупно
    font_which = load_font(font_oswald, 130)
    draw.text((50, 450), "КОТОРЫЕ", font=font_which, fill=text_color)
    
    # Месяц вертикально справа
    font_month = load_font(font_bebas, 90)
    month_upper = month.upper()
    month_img = Image.new('RGBA', (200, 600), (0, 0, 0, 0))
    month_draw = ImageDraw.Draw(month_img)
    month_draw.text((0, 0), month_upper, font=font_month, fill=text_color)
    month_rotated = month_img.rotate(90, expand=True)
    img.paste(month_rotated, (width - 150, 300), month_rotated)
    
    # "СТОИТ" - Bebas Neue
    font_worth = load_font(font_bebas, 100)
    draw.text((300, 600), "СТОИТ", font=font_worth, fill=text_color)
    
    # "ПОСЕТИТЬ" - Druk Cyr Bold
    font_visit = load_font(font_druk, 90)
    draw.text((50, 750), "ПОСЕТИТЬ", font=font_visit, fill=text_color)
    
    output_path = Path(output_path)
    img.save(str(output_path), 'PNG')
    log(f"✅ Cover saved: {output_path} ({output_path.stat().st_size} bytes)")
    return output_path

# Тест генерации обложки
test_cover = WORKING_DIR / "test_cover.png"
generate_cover_image("25-27", "ДЕКАБРЯ", test_cover)
log("✅ Cover generator ready")

In [ ]:
# ==========================================
# 4. BLENDER XPBD SCRIPT (встроенный)
# ==========================================
import base64

# Полный blender_xpbd_paper.py (780 строк) закодирован в base64
BLENDER_SCRIPT_B64 = """..."""

# Декодируем и сохраняем скрипт
BLENDER_SCRIPT_PATH = WORKING_DIR / "blender_xpbd_paper.py"
BLENDER_SCRIPT_PATH.write_bytes(base64.b64decode(BLENDER_SCRIPT_B64))
log(f"✅ Embedded Blender XPBD script written: {BLENDER_SCRIPT_PATH} ({BLENDER_SCRIPT_PATH.stat().st_size} bytes)")

def run_blender_render(poster_path: Path, out_dir: Path) -> bool:
    """Run Blender in background to render frames."""
    if not BLENDER_SCRIPT_PATH.exists():
        log(f"❌ Blender script not found: {BLENDER_SCRIPT_PATH}")
        return False
    
    out_dir.mkdir(parents=True, exist_ok=True)
    
    cmd = [
        "blender", "-b", "--factory-startup", "-noaudio",
        "-P", str(BLENDER_SCRIPT_PATH),
        "--",
        "--poster", str(poster_path),
        "--out_dir", str(out_dir),
        "--frame_start", str(FRAME_START),
        "--frame_end", str(FRAME_END),
        "--samples", str(BLENDER_SAMPLES)
    ]
    log(f"🎬 Rendering: {poster_path.name}")
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, timeout=1800)
        if result.returncode != 0:
            log(f"❌ Blender error (exit {result.returncode}):")
            log(result.stderr[:1000] if result.stderr else "No stderr")
            return False
    except subprocess.TimeoutExpired:
        log("❌ Blender timeout (30 min)")
        return False
    except Exception as e:
        log(f"❌ Blender exception: {e}")
        return False
    
    # Проверяем что кадры созданы
    frames = list(out_dir.glob("frame_*.png"))
    if not frames:
        log("❌ No frames generated")
        return False
    
    # Проверяем размер первого кадра
    first_frame = sorted(frames)[0]
    frame_size = first_frame.stat().st_size
    if frame_size < 10000:
        log(f"⚠️ Frame too small ({frame_size} bytes)")
    
    log(f"✅ Done: {len(frames)} frames, first={frame_size/1024:.1f}KB")
    return True

log("✅ Blender runner ready")


In [ ]:
# ==========================================
# 5. SHOWREEL ASSEMBLY (из make_showreel_audio.py)
# ==========================================
from dataclasses import dataclass
import math

def blend(a, b, t: float):
    """Blend two frames with weight t (0.0 = a, 1.0 = b)"""
    t = float(max(0.0, min(1.0, t)))
    return cv2.addWeighted(a, 1.0 - t, b, t, 0.0)

def ease_linear(x: float) -> float:
    return x

def ease_in_quad(x: float) -> float:
    return x * x

def ease_out_quad(x: float) -> float:
    return 1.0 - (1.0 - x) * (1.0 - x)

def ease_in_out_cubic(x: float) -> float:
    if x < 0.5:
        return 4.0 * x * x * x
    return 1.0 - pow(-2.0 * x + 2.0, 3.0) / 2.0

def ease_out_expo(x: float) -> float:
    return 1.0 if x >= 1.0 else 1.0 - pow(2, -10 * x)

def ease_in_out_back(x: float) -> float:
    c1 = 1.70158
    c2 = c1 * 1.525
    if x < 0.5:
        return (pow(2 * x, 2) * ((c2 + 1) * 2 * x - c2)) / 2
    return (pow(2 * x - 2, 2) * ((c2 + 1) * (x * 2 - 2) + c2) + 2) / 2

EASING = {
    "linear": ease_linear,
    "ease_in_quad": ease_in_quad,
    "ease_out_quad": ease_out_quad,
    "ease_in_out_cubic": ease_in_out_cubic,
    "ease_out_expo": ease_out_expo,
    "ease_in_out_back": ease_in_out_back,
}

def render_motion(base: list, length: int, easing_name: str, reverse: bool = False, frame_interp: bool = True) -> list:
    """Render motion with proper easing and frame interpolation."""
    if length <= 0 or not base:
        return []
    if len(base) == 1:
        return [base[0]] * int(length)
    
    ease = EASING.get(easing_name, ease_in_out_cubic)
    out = []
    n = max(1, int(length))
    
    for i in range(n):
        t = 0.0 if n == 1 else (i / (n - 1))
        s = float(max(0.0, min(1.0, ease(t)))) * (len(base) - 1)
        if reverse:
            s = (len(base) - 1) - s
        
        if not frame_interp:
            idx = int(round(s))
            out.append(base[idx])
        else:
            i0 = int(math.floor(s))
            i1 = min(len(base) - 1, i0 + 1)
            alpha = float(s - i0)
            out.append(blend(base[i0], base[i1], alpha))
    
    return out

# Timing parameters from original make_showreel_audio.py
UNFOLD_LEN = 24
FOLD_LEN = 24
HOLD_FLAT = 30
HOLD_BALL = 2
FRAME_INTERP = True

@dataclass
class SegmentPlan:
    stem: str
    unfold_len: int = UNFOLD_LEN
    fold_len: int = FOLD_LEN
    hold_flat: int = HOLD_FLAT
    hold_ball: int = HOLD_BALL
    easing_unfold: str = "ease_in_out_back"
    easing_fold: str = "ease_out_quad"

def composite_rgba_on_bg(rgba_frame, bg_color=(20, 20, 25)):
    """Composite RGBA frame onto dark gradient background."""
    h, w = rgba_frame.shape[:2]
    bg = np.zeros((h, w, 3), dtype=np.uint8)
    for y in range(h):
        factor = 0.8 + 0.2 * (y / h)
        bg[y, :] = [int(c * factor) for c in bg_color]
    
    if rgba_frame.shape[2] == 4:
        alpha = rgba_frame[:, :, 3:4].astype(np.float32) / 255.0
        rgb = rgba_frame[:, :, :3].astype(np.float32)
        result = (rgb * alpha + bg.astype(np.float32) * (1 - alpha))
        return result.astype(np.uint8)
    else:
        return rgba_frame[:, :, :3]

def load_segment_frames(frames_dir: Path, frame_start: int, frame_end: int) -> list:
    """Load and composite Blender frames for a segment."""
    frames = []
    for fn in range(frame_start, frame_end + 1):
        frame_path = frames_dir / f"frame_{fn:04d}.png"
        if not frame_path.exists():
            continue
        img = cv2.imread(str(frame_path), cv2.IMREAD_UNCHANGED)
        if img is None:
            continue
        composited = composite_rgba_on_bg(img)
        frames.append(cv2.cvtColor(composited, cv2.COLOR_BGR2RGB))
    return frames

log("✅ Showreel functions ready")



In [ ]:
# ==========================================
# 6. AUDIO & VIDEO ASSEMBLY
# ==========================================

def add_audio(video_path: Path, audio_path: Path, output_path: Path, start_sec: float = 70) -> bool:
    """Mix audio starting at specified timestamp with video."""
    if not audio_path.exists():
        log(f"⚠️ Audio not found: {audio_path}")
        shutil.copy(video_path, output_path)
        return True
    
    cmd = [
        "ffmpeg", "-y",
        "-ss", str(float(start_sec)),
        "-i", str(audio_path),
        "-i", str(video_path),
        "-shortest",
        "-c:v", "copy",
        "-c:a", "aac",
        "-b:a", "192k",
        str(output_path)
    ]
    result = subprocess.run(cmd, capture_output=True)
    if result.returncode == 0:
        log(f"✅ Audio added: {output_path}")
        return True
    else:
        log(f"❌ FFmpeg audio error")
        return False

def render_video_from_frames(frames_dir: Path, output_path: Path, fps: int = 24) -> bool:
    """Use FFmpeg to assemble frames into video."""
    cmd = [
        "ffmpeg", "-y",
        "-framerate", str(fps),
        "-i", str(frames_dir / "frame_%06d.png"),
        "-c:v", "libx264",
        "-pix_fmt", "yuv420p",
        "-crf", "18",
        str(output_path)
    ]
    result = subprocess.run(cmd, capture_output=True)
    if result.returncode == 0:
        log(f"✅ Video rendered: {output_path} ({output_path.stat().st_size / 1024:.1f} KB)")
        return True
    else:
        log(f"❌ FFmpeg video error")
        return False

log("✅ Audio/Video functions ready")

In [ ]:
# ==========================================
# 7. ОСНОВНОЙ ПАЙПЛАЙН
# ==========================================

def main_pipeline():
    """Full pipeline."""
    
    payload_path = SOURCE_FOLDER / "payload.json"
    if not payload_path.exists():
        log(f"ERROR: payload.json not found at {payload_path}")
        return False
    
    payload = json.loads(payload_path.read_text())
    scenes = payload.get("scenes", [])
    
    cover_path = WORKING_DIR / "cover.png"
    generate_cover_image("25-27", "DECEMBER", cover_path)
    
    posters = [cover_path]
    for i, scene in enumerate(scenes[:4]):
        # Try to find poster files
        for ext in [".jpg", ".png", ".jpeg"]:
            local = SOURCE_FOLDER / f"poster{i+1}{ext}"
            if local.exists():
                posters.append(local)
                break
    
    log(f"Posters to render: {len(posters)}")
    
    # Build segments
    segments = []
    segments.append(SegmentPlan(
        stem="frames_0",
        unfold_len=0,
        fold_len=24,
        hold_flat=36,
        hold_ball=2,
        easing_unfold="linear",
        easing_fold="ease_out_quad"
    ))
    
    for i in range(1, len(posters)):
        segments.append(SegmentPlan(
            stem=f"frames_{i}",
            unfold_len=28,
            fold_len=28,
            hold_flat=30,
            hold_ball=2,
            easing_unfold="ease_in_out_back",
            easing_fold="ease_in_out_back"
        ))
    
    # Render Blender frames
    for i, poster in enumerate(posters):
        frames_dir = WORKING_DIR / f"frames_{i}"
        success = run_blender_render(poster, frames_dir)
        if not success:
            log(f"Skipping poster {i}")
    
    # Assemble sequence
    log("Assembling showreel...")
    sequence = []
    
    for seg in segments:
        frames_dir = WORKING_DIR / seg.stem
        frames = load_segment_frames(frames_dir, FRAME_START, FRAME_END)
        
        if not frames:
            log(f"No frames for {seg.stem}")
            continue
        
        ball = frames[0]
        flat = frames[-1]
        
        if seg.unfold_len > 0:
            unfold = render_motion(frames, seg.unfold_len, seg.easing_unfold, False, True)
            sequence.extend(unfold)
        
        for _ in range(seg.hold_flat):
            sequence.append(flat)
        
        fold = render_motion(frames, seg.fold_len, seg.easing_fold, True, True)
        sequence.extend(fold)
        
        for _ in range(seg.hold_ball):
            sequence.append(ball)
    
    log(f"Total frames: {len(sequence)}")
    
    if not sequence:
        log("FATAL: No frames!")
        return False
    
    seq_dir = WORKING_DIR / "sequence"
    seq_dir.mkdir(exist_ok=True)
    
    for i, frame in enumerate(sequence):
        frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        cv2.imwrite(str(seq_dir / f"frame_{i+1:06d}.png"), frame_bgr)
    
    video_path = WORKING_DIR / "crumple_video.mp4"
    render_video_from_frames(seq_dir, video_path, fps=24)
    
    final_path = WORKING_DIR / "crumple_video_final.mp4"
    audio_path = list(SOURCE_FOLDER.glob("*.mp3"))
    if audio_path:
        add_audio(video_path, audio_path[0], final_path)
    else:
        shutil.copy(video_path, final_path)
    
    if final_path.exists():
        size_kb = final_path.stat().st_size / 1024
        log(f"Final video: {size_kb:.1f} KB")
        if size_kb < 500:
            log("FAIL: Video too small!")
            return False
        return True
    
    return False

log("=" * 50)
log("Starting CrumpleVideo Pipeline")
log("=" * 50)

success = main_pipeline()

if success:
    log("Pipeline completed!")
else:
    log("Pipeline failed!")

